In [1]:
import torch
torch.cuda.set_device(7)
if torch.cuda.is_available():
    current_gpu = torch.cuda.current_device()
    print(f"Current default GPU index: {current_gpu}")
    print(f"Current default GPU name: {torch.cuda.get_device_name(current_gpu)}")
else:
    print("No GPUs available.")

Current default GPU index: 7
Current default GPU name: NVIDIA A40


In [2]:
from PIL import Image
import requests
from transformers import AutoProcessor, LlavaForConditionalGeneration
from tqdm import tqdm

In [3]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
from transformers import pipeline

model_id = "llava-hf/llava-1.5-7b-hf"

pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
image_path = "saved_data/cifar_test100/apple7.png"
class_name = 'apple'
# prompt = "USER: <image>\nGenerate a detailed and concise description less than 77 words of the bee in this image and the overall mood of this image. Focus on major colors, notable objects, and any distinct atmosphere or emotion it conveys.\nASSISTANT:"
prompt = f"USER: <image>\nGenerate a concise and vivid description less than 77 words using exactly the word '{class_name}' to describe the '{class_name}' in this image. Describe the major features such as color and shape of the {class_name} in the image. Do not use synonyms or related terms for the main item but only using '{class_name}' to refer it.\nASSISTANT:"

image = Image.open(image_path)
outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 150})
print(outputs[0]["generated_text"])

USER:  
Generate a concise and vivid description less than 77 words using exactly the word 'apple' to describe the 'apple' in this image. Describe the major features such as color and shape of the apple in the image. Do not use synonyms or related terms for the main item but only using 'apple' to refer it.
ASSISTANT: A red apple with a stem in the center.


In [5]:
outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
print(outputs[0]["generated_text"])

USER:  
Generate a concise and vivid description less than 77 words using exactly the word 'apple' to describe the 'apple' in this image. Describe the major features such as color and shape of the apple in the image. Do not use synonyms or related terms for the main item but only using 'apple' to refer it.
ASSISTANT: A red apple with a stem in the center.


In [6]:

def extract_classname(image_path):
    # Split the path into parts on the '/'
    parts = image_path.split('/')
    # The last part is "apple0.png", so we take the last element
    filename = parts[-1]
    # Now we need to remove the number and extension, assuming the format is always classname + number + .png
    classname = ''.join([char for char in filename if not char.isdigit()]).replace('.png', '')
    return classname

def extract_classname(image_path):
    # Split the path into parts on the '/'
    parts = image_path.split('/')
    # The last part is the filename, e.g., "apple0.png"
    filename = parts[-1]
    # Remove the number and extension, handling both .png and .jpeg/.jpg formats
    if filename.lower().endswith('.png'):
        classname = filename[:-4]  # Remove '.png'
    elif filename.lower().endswith('.jpeg'):
        classname = filename[:-5]  # Remove '.jpeg'
    elif filename.lower().endswith('.jpg'):
        classname = filename[:-4]  # Remove '.jpg'
    else:
        classname = filename  # If no recognized extension, use the filename as is

    # Remove any digits from the remaining part of the filename
    classname = ''.join([char for char in classname if not char.isdigit()])
    return classname


def llava_single_pipeline(image_path, prompt):
    image = Image.open(image_path)
    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 77})
    full_response = outputs[0]["generated_text"]
    # print(full_response)
    assistant_index = full_response.find("ASSISTANT:") + len("ASSISTANT:")
    caption = full_response[assistant_index:].strip()
    return caption


def process_images(input_file_path, output_file_path, user_input):
    with open(input_file_path, 'r') as file:
        image_urls = file.readlines()

    with open(output_file_path, 'w') as output_file:
        for n, line in enumerate(image_urls):
        # for line in tqdm(image_urls, desc="Processing Images", leave=False):
            image_url = line.split()[0]
            image_url = image_url.strip()  # Remove any extra whitespace
            output = llava_single_pipeline(image_url, user_input)
            if n % 10 == 0:
                print(n)
                print(output)
            output_file.write(f"{image_url}: {output}\n")


# load dataset

In [7]:
import torch
import os
from torch import cat, Tensor
from torch.nn import Module
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torchvision import datasets, transforms
import torch.optim.lr_scheduler # ?
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop, RandomHorizontalFlip, Resize
from torchvision.transforms.functional import center_crop
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.utils import save_image
from torchvision.transforms.functional import pil_to_tensor

In [19]:
import json

def read_json(json_path):
# Step 1: Open the JSON file
    with open(json_path, 'r') as file:
    # Step 2: Load the JSON data
        data = json.load(file)
    return data

name_dict = read_json('/scratch/local/ssd/enbo/saved_data/imagenet_label_name.json')
name_list = [item for key, item in name_dict.items()]
print(name_list)

['chambered_nautilus', 'harvestman', 'macaw', 'bittern', 'electric_ray', 'drake', 'agama', 'night_snake', 'indigo_bunting', 'tiger_shark', 'flamingo', 'garter_snake', 'common_newt', 'tench', 'sulphur-crested_cockatoo', 'white_stork', 'terrapin', 'diamondback', 'flatworm', 'sea_snake', 'spiny_lobster', 'green_snake', 'bald_eagle', 'kite', 'goldfish', 'scorpion', 'goose', 'bulbul', 'chickadee', 'sea_anemone', 'magpie', 'mud_turtle', 'crayfish', 'lorikeet', 'garden_spider', 'spoonbill', 'hermit_crab', 'great_white_shark', 'green_mamba', 'bee_eater', 'bustard', 'Komodo_dragon', 'hognose_snake', 'common_iguana', 'king_snake', 'toucan', 'peacock', 'barn_spider', 'wolf_spider', 'thunder_snake', 'water_ouzel', 'Dungeness_crab', 'nematode', 'vine_snake', 'wombat', 'black_widow', 'oystercatcher', 'black_grouse', 'red-backed_sandpiper', 'goldfinch', 'snail', 'hammerhead', 'spotted_salamander', 'American_alligator', 'banded_gecko', 'wallaby', 'great_grey_owl', 'jellyfish', 'black_swan', 'ptarmigan

In [20]:
# import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import os

# stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

# transform = transform_train = Compose([
#     # Resize(224),
#     # Resize(384),
#     # RandomHorizontalFlip(),
#     ToTensor(),
#     # Normalize(*stats,inplace=True)
# ])

# # Load the CIFAR-100 training set
# trainset = torchvision.datasets.CIFAR100(root='data', train=True,download=True, transform=transform)

# name_list = trainset.classes
# print(name_list)

# synthetic classes

In [23]:
order_list = [[36, 0, 54, 5, 20],
 [22, 45, 13, 83, 19],
 [26, 73, 16, 62, 33],
 [34, 98, 24, 74, 53],
 [10, 94, 51, 4, 32],
 [38, 81, 50, 40, 41],
 [30, 89, 69, 64, 21],
 [84, 14, 88, 49, 68],
 [6, 80, 57, 65, 46],
 [9, 91, 48, 72, 31],
 [76, 7, 47, 8, 1],
 [61, 75, 63, 18, 86],
 [59, 70, 43, 85, 95],
 [27, 93, 35, 25, 82],
 [44, 56, 67, 66, 37],
 [60, 11, 2, 78, 52],
 [97, 39, 55, 3, 99],
 [29, 71, 23, 28, 90],
 [87, 15, 92, 17, 77],
 [12, 42, 96, 79, 58]]

syn_classes = [order[:3] for order in order_list]
syn_classes = [item for lists in syn_classes for item in lists]
real_classes = list(set([i for i in range(100)]) - set(syn_classes))
print(len(real_classes))

40


In [24]:
prompt = "USER: <image>\nGenerate a detailed and concise description less than 77 words of the bee in this image and the overall mood of this image. Focus on major colors, notable objects, and any distinct atmosphere or emotion it conveys.\nASSISTANT:"
class_name = 'apple'
caption_prompt = f"USER: <image>\nGenerate a long, detailed and concise description of about 77 words using exactly the word '{class_name}' to describe the {class_name} in this image. Focus on major colors, notable objects, and any distinct atmosphere or emotion the image conveys. Do not use synonyms or related terms for the main item but only using '{class_name}' to refer it.\nASSISTANT:"
print(caption_prompt)

USER: <image>
Generate a long, detailed and concise description of about 77 words using exactly the word 'apple' to describe the apple in this image. Focus on major colors, notable objects, and any distinct atmosphere or emotion the image conveys. Do not use synonyms or related terms for the main item but only using 'apple' to refer it.
ASSISTANT:


In [25]:
# old_caption_prompt = f"USER: <image>\nGenerate a long, detailed and concise description of about 77 words using exactly the word '{class_name}' to describe the {class_name} in this image. Focus on major colors, notable objects, and any distinct atmosphere or emotion the image conveys. Do not use synonyms or related terms for the main item but only using '{class_name}' to refer it.\nASSISTANT:"

caption_prompt = f"USER: <image>\nGenerate a concise description of about 15 words using exactly the word '{class_name}' to describe the {class_name} in this image. Focus on major colors, notable objects in the image. Do not use synonyms or related terms for the main item but only using '{class_name}' to refer it.\nASSISTANT:"
caption_prompt = f"USER: <image>\nGenerate a long, detailed and concise description of about 77 words using exactly the word '{class_name}' to describe the {class_name} in this image. Focus on major colors, notable objects, and any distinct atmosphere or emotion the image conveys. Do not use synonyms or related terms for the main item but only using '{class_name}' to refer it.\nASSISTANT:"


In [26]:
def check_processed_classes(output_dir, expected_line_count):
    # List all files in the output directory
    existing_files = set(os.listdir(output_dir))
    # Find all class numbers that have been processed
    processed_classes = []
    for file in existing_files:
        if file.startswith('class') and file.endswith('.txt'):
            class_number = int(file.split('class')[1].split('.txt')[0])
            file_path = os.path.join(output_dir, file)
            
            # Check the number of lines in the file
            with open(file_path, 'r') as f:
                line_count = sum(1 for _ in f)
            
            if line_count == expected_line_count:
                processed_classes.append(class_number)
    
    return processed_classes
    
re = check_processed_classes('/storage3/enbo/saved_data/llava_imagenet/imagenet_short_60real/', 1300)
# re
rest = list(set(real_classes)-set(re)) 
rest

[1,
 3,
 4,
 5,
 8,
 17,
 18,
 19,
 20,
 21,
 25,
 28,
 31,
 32,
 33,
 37,
 40,
 41,
 46,
 49,
 52,
 53,
 58,
 62,
 64,
 65,
 66,
 68,
 72,
 74,
 77,
 78,
 79,
 82,
 83,
 85,
 86,
 90,
 95,
 99]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
image_save_dir = '/scratch/local/ssd/enbo/saved_data/imagenet_train_data_allimages'

# for i in real_classes:
for i in tqdm(real_classes, desc="Processing Each Class"):
    class_name = name_list[i]
    print(i, class_name)
    current_class_txt_path = os.path.join(image_save_dir, f"class{i}.txt")
    # print(current_class_txt_path)
    output_dict = '/storage3/enbo/saved_data/llava_imagenet/imagenet_long_40syn'
    if not os.path.exists(output_dict):
        os.makedirs(output_dict)
    output_path = os.path.join(output_dict, f'class{i}.txt')
    caption_prompt = f"USER: <image>\nGenerate a detailed and concise description less than 77 words of the {class_name} in this image and the overall mood of this image. Focus on major colors, notable objects, and any distinct atmosphere or emotion it conveys. Do not use synonyms or related terms for the main item but only using '{class_name}' to refer it.\nASSISTANT:"
    # caption_prompt = f"USER: <image>\nGenerate a concise and vivid description using exactly the word '{class_name}' to describe the '{class_name}' in this image. Describe the major features including shape, color of the notable objects in the image. Do not use synonyms or related terms for the main item but only using '{class_name}' to refer it.\nASSISTANT:"

    process_images(current_class_txt_path, output_path, caption_prompt)

Processing Each Class:   0%|                                      | 0/40 [00:00<?, ?it/s]

1 harvestman
0
The harvestman is a large spider with a red abdomen and long legs. It is sitting on a rusted metal surface, possibly a pipe or a drain cover. The overall mood of the image is eerie and somewhat unsettling, as the harvestman appears to be a large and intimidating creature. The combination of the spider
10
The image features a harvestman, a spider-like creature, perched on a purple flower. The harvestman is large and has long legs, making it an interesting focal point in the scene. The overall mood of the image is serene and peaceful, with the harvestman seemingly at ease in its natural habitat. The vibrant
20
The image features a harvestman, a spider-like creature, sitting on a tree branch. The harvestman is the main focus of the scene, and its presence creates a sense of curiosity and fascination. The overall mood of the image is one of wonder and discovery, as the viewer gets a glimpse into the world of these unique creatures.
30
The image features a harvestman sitting 

Processing Each Class:   2%|▌                      | 1/40 [1:17:25<50:19:20, 4645.15s/it]

3 bittern
0
The image features a bittern standing in a body of water, possibly a river or a pond. The bird is surrounded by tall grass, creating a serene and natural atmosphere. The water appears to be calm, and the bittern seems to be enjoying its time in the water. The scene captures the beauty of nature and the harmony between the bird and its environment
10
The bittern is standing in a body of water, possibly a pond or a lake. It has a long neck and a distinctive appearance, with its head tilted upwards. The overall mood of the image is calm and serene, as the bittern appears to be undisturbed by its surroundings. The water appears to be still, and the
20
The image features a bittern swimming in a body of water, surrounded by tall grass. The water appears to be murky, and the bittern seems to be enjoying its time in the water. The overall mood of the image is serene and peaceful, as the bittern swims undisturbed in its natural habitat.
30
The bittern in the image is a large bird wi

Processing Each Class:   5%|█▏                     | 2/40 [2:13:55<41:14:19, 3906.83s/it]

...

79 loggerhead
0
The image features a loggerhead turtle laying on the beach, surrounded by foamy water. The turtle appears to be resting or possibly preparing to enter the water. The overall mood of the image is serene and peaceful, as it captures a quiet moment in nature. The presence of the turtle and the gentle sound of the waves
10
The image features a loggerhead turtle swimming in a tank filled with water and surrounded by various fish. The turtle is the main focus of the scene, with its distinctive shell and body shape. The overall mood of the image is calm and serene, as the turtle and fish coexist peacefully in their aquatic environment.
20
The image features a loggerhead turtle lying on the sandy beach. The turtle is the main focus of the scene, and its presence creates a serene and peaceful atmosphere. The turtle is surrounded by a few people, who seem to be observing or taking care of the turtle. The overall mood of the image is calm and relax
30
The loggerhead is a large, co

Processing Each Class:  82%|██████████████████▏   | 33/40 [32:28:03<6:40:24, 3432.03s/it]

82 tick
0
The tick in the image is brown and has a distinctive appearance. It is captured in a close-up shot, showcasing its intricate details. The overall mood of the image is somewhat eerie, as the tick is a parasite that can cause discomfort or even illness to its host. The close-up view of the tick highlights its
10
The tick is brown and black, with a yellow spot on its back. It is sitting on a person's skin, possibly on their neck. The overall mood of the image is unsettling, as ticks are often associated with discomfort and the feeling of being invaded by a foreign creature.
20
The tick in the image is small and red, with a brown spot on its back. It is resting on a pink surface, which could be a person's hand or a piece of fabric. The overall mood of the image is somewhat unsettling, as ticks are often associated with diseases and discomfort. The close-up view of the tick
30
The image features a close-up of a tick on a mouse's ear. The tick is brown and appears to be a significa

Processing Each Class:  85%|██████████████████▋   | 34/40 [33:25:40<5:43:56, 3439.44s/it]

83 sea_lion
0
The image features a group of sea lions resting on a dock. There are at least 11 sea lions in the scene, with some lying down and others sitting up. The sea lions are spread across the dock, with some closer to the water and others further back. The overall mood of the image is peaceful and serene, as the sea l
10
The image features a group of sea lions resting on rocks. There are at least 11 sea lions in the scene, with some lying down and others sitting or standing. The rocks are located near the water, providing a natural habitat for the sea lions. The overall mood of the image is serene and peaceful, as the sea lions seem to
20
The image features two sea lions, one larger and one smaller, standing close to each other on a beach. The larger sea lion is gently nuzzling its head against the smaller one, creating a tender and affectionate moment. The scene captures the beauty of these majestic creatures in their natural habitat, showcasing their social behavior and bondin

In [17]:
def count_txt_files(directory):
    """
    Counts the number of .txt files in the specified directory.

    Args:
    directory (str): The path to the directory to search for .txt files.

    Returns:
    int: The number of .txt files in the directory.
    """
    txt_count = 0
    # List all files and directories in the specified directory
    for entry in os.listdir(directory):
        # Construct full path
        full_path = os.path.join(directory, entry)
        # Check if it's a file with a .txt extension
        if os.path.isfile(full_path) and entry.endswith('.txt'):
            txt_count += 1
    
    return txt_count

output_folder = '/scratch/local/ssd/enbo/saved_data/llava_imagenet/imagenet_long_60real'
count = count_txt_files(output_folder)
print(f"There are {count} .txt files in the folder.")
# count = count_png_files(output_folder)
# print(f"There are {count} .png files in the folder.")

There are 60 .txt files in the folder.


In [16]:
import os

# Specify the directory path
directory_path = 'saved_data/cifar_train_all_fortest'

# Loop through the directory and print out .txt files with line counts
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r') as file:
            line_count = sum(1 for line in file)
        print(f"{filename}: {line_count} lines")



class84.txt: 500 lines
class6.txt: 500 lines
class41.txt: 500 lines
class38.txt: 500 lines
class22.txt: 500 lines
class87.txt: 500 lines
class57.txt: 500 lines
class86.txt: 500 lines
class8.txt: 500 lines
class20.txt: 500 lines
class62.txt: 500 lines
class17.txt: 500 lines
class25.txt: 500 lines
class24.txt: 500 lines
class7.txt: 500 lines
class73.txt: 500 lines
class64.txt: 500 lines
class88.txt: 500 lines
class98.txt: 500 lines
class96.txt: 500 lines
class26.txt: 500 lines
class31.txt: 500 lines
class36.txt: 500 lines
class32.txt: 500 lines
class63.txt: 500 lines
class37.txt: 500 lines
class19.txt: 500 lines
class5.txt: 500 lines
class18.txt: 500 lines
class99.txt: 500 lines
class0.txt: 500 lines
class75.txt: 500 lines
class40.txt: 500 lines
class23.txt: 500 lines
class97.txt: 500 lines
class51.txt: 500 lines
class44.txt: 500 lines
class47.txt: 500 lines
class1.txt: 500 lines
class74.txt: 500 lines
class14.txt: 500 lines
class83.txt: 500 lines
class9.txt: 500 lines
class21.txt: 500 l

In [61]:
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
